# Loading MODIS data from the datacube

This notebook will briefly discuss how to load MODIS data from the datacube. It is a duplicate of the Landsat notebook modified only to use MODIS instead.

**Requirements:**
* A running data cube created via 01 - Getting Started

In [1]:
# This next line enables interactive plots for use later in the notebook
%matplotlib notebook

## Importing the datacube module

To start with, we'll import the datacube module and load an instance of the datacube and call our application name *load-data-example*. The application name can be anything, its used in logging information so you can trace results etc using the name.
We'll keep this example satisfying in that you can see some data quickly. Later as we look at more specific products and analysis we'll introduce more variants of the loading process.

In [2]:
import datacube
dc = datacube.Datacube(app='load-data-example')


# List metadata for all Landsat 
dc_products = dc.list_products()
display_columns = ['name', 'description', 'product_type', 'crs', 'resolution', 'spatial_dimensions']
dc_products[dc_products['name'].str.contains("modis")][display_columns].set_index('name')

/home/jovyan/.local/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


,description,product_type,crs,resolution,spatial_dimensions
name,,,,,
modis_mcd43a1_tile,MODIS 500 metre MCD43A1 Collection 006,MCD43A1,"PROJCS[""unnamed"",GEOGCS[""Unknown datum based u...",NaN,"(y, x)"
modis_mcd43a2_tile,MODIS 500 metre MCD43A2 Collection 006,MCD43A2,"PROJCS[""unnamed"",GEOGCS[""Unknown datum based u...",NaN,"(y, x)"
modis_mcd43a3_tile,MODIS 500 metre MCD43A3 Collection 006,MCD43A3,"PROJCS[""unnamed"",GEOGCS[""Unknown datum based u...",NaN,"(y, x)"
modis_mcd43a4_tile,MODIS 500 metre MCD43A4 Collection 006,MCD43A4,"PROJCS[""unnamed"",GEOGCS[""Unknown datum based u...",NaN,"(y, x)"


In [4]:
# Define temporal and spatial extents to pass to `dc.load` for the data extraction:
query = {'lat': (-35.25, -35.35),
         'lon': (149.05, 149.17),
         'time':('2018-09-01', '2018-09-01'),
         'output_crs': 'epsg:3577',
         'resolution': (-500,500)}

In [5]:
data = dc.load(product='modis_mcd43a4_tile', **query, group_by='solar_day')


In [6]:
data


<xarray.Dataset>
Dimensions:                                   (time: 4, x: 26, y: 26)
Coordinates:
  * time                                      (time) datetime64[ns] 2018-08-24T23:59:59.999999 ... 2018-09-05T23:59:59.999999
  * y                                         (y) float64 -3.953e+06 ... -3.966e+06
  * x                                         (x) float64 1.542e+06 ... 1.555e+06
Data variables:
    BRDF_Albedo_Band_Mandatory_Quality_Band1  (time, y, x) uint8 1 1 1 ... 1 1 0
    BRDF_Albedo_Band_Mandatory_Quality_Band2  (time, y, x) uint8 0 1 1 ... 1 1 0
    BRDF_Albedo_Band_Mandatory_Quality_Band3  (time, y, x) uint8 1 1 1 ... 1 1 0
    BRDF_Albedo_Band_Mandatory_Quality_Band4  (time, y, x) uint8 1 1 1 ... 1 1 0
    BRDF_Albedo_Band_Mandatory_Quality_Band5  (time, y, x) uint8 0 1 1 ... 1 1 0
    BRDF_Albedo_Band_Mandatory_Quality_Band6  (time, y, x) uint8 1 1 1 ... 1 1 0
    BRDF_Albedo_Band_Mandatory_Quality_Band7  (time, y, x) uint8 1 1 1 ... 1 1 0
    Nadir_Reflectance_Ban

In [7]:
from datacube.storage import masking

# Set all nodata pixels to `NaN`:
data = masking.mask_invalid_data(data)
# Set all invalid data to `NaN` - valid range for USRS SR is 0 to 10000, but the surface reflectance product can have values just outside this range
# We remove them so the image drawn isn't impacted by them
data = data.where((data >= 0) & (data<=10000))

# Select a time slice from the EO data and combine the bands into a 3 band array
image_array = data[['Nadir_Reflectance_Band1', 'Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band3']].isel(time=0).to_array()
# Show the image
image_array.plot.imshow(robust=True, figsize=(8, 8))


KeyError: 'red'

More information on the load function see the API docs: :py:meth:`~datacube.Datacube.load`

# Show/Hide error information
Yep, here is it again in case you need it

In [6]:
from IPython.display import HTML

HTML('''<script>
code_show_err=false; 
function code_toggle_err() {
 if (code_show_err){
 $('div.output_error').hide();
 $('div.output_stderr').hide();
 } else {
 $('div.output_error').show();
 $('div.output_stderr').show();
 }
 code_show_err = !code_show_err
} 
$( document ).ready(code_toggle_err);
</script>
<form action="javascript:code_toggle_err()"><input type="submit" value="Click here to toggle on/off the error output."></form>''')
